# **Computational Theory Assessment (2024/2025)**
Ronan Francis (G00403092)
Computational Theory

In [68]:
import math
import string
import random
import struct
import hashlib

## **Table of Contents**
1. [Task 1: Binary Representations](#task-1-binary-representations)
2. [Task 2: Hash Functions](#task-2-hash-functions)
3. [Task 3: SHA256 Padding](#task-3-sha256-padding)
4. [Task 4: Prime Numbers](#task-4-prime-numbers)
5. [Task 5: Roots](#task-5-roots)
6. [Task 6: Proof of Work](#task-6-proof-of-work)
7. [Task 7: Turing Machines](#task-7-turing-machines)
8. [Task 8: Computational Complexity](#task-8-computational-complexity)

---

# **Task 1: Binary Representations**

## Introduction 
In this task, we focus on implementing several core functions that manipulate binary representations of 32-bit unsigned integers. These operations are crucial in many fields, including cryptography (like SHA-256), data encoding, and low-level programming.

### Circular Shifts

Circular shifts (or bit rotations) involve shifting the bits of a number to the left or right, with the bits that "fall off" one end reappearing on the opposite end. This operation is distinct from logical shifts, where bits shifted off one end are simply discarded and zeros are introduced on the other end. Circular shifts maintain the bit-length and the overall "weight" of the data.

For example, when you perform a left circular shift (`rotl`) on a 32-bit integer:
- The bits are moved to the left by a specified number of positions.
- The bits that move past the most significant bit are wrapped around to the least significant bit positions.

Similarly, a right circular shift (`rotr`) moves bits to the right with wrapping from the rightmost bits to the leftmost positions.

For more detailed information, see the [Circular Shift Wikipedia article](https://en.wikipedia.org/wiki/Circular_shift).

### Bitwise Operators

Python provides several bitwise operators that allow for efficient manipulation of integers at the binary level:

- **AND (`&`):** Compares each bit of two integers and returns `1` only if both bits are `1`.
- **OR (`|`):** Compares each bit of two integers and returns `1` if at least one of the bits is `1`.
- **XOR (`^`):** Returns `1` for each bit position where the corresponding bits of the two operands are different.
- **NOT (`~`):** Inverts the bits of the operand.
- **Left Shift (`<<`):** Shifts the bits of a number to the left by a specified number of positions.
- **Right Shift (`>>`):** Shifts the bits of a number to the right by a specified number of positions.

These operators form the foundation for our binary manipulation functions. Detailed explanations and examples can be found at [GeeksforGeeks Bitwise Operators](https://www.geeksforgeeks.org/python-bitwise-operators/).

In summary, Task 1 involves implementing critical binary operations using circular shifts and bitwise operators. These functions serve as building blocks for more complex algorithms like SHA-256 and are widely applicable in many areas of computer science. By understanding and implementing `rotl`, `rotr`, `ch`, and `maj`, you not only gain proficiency in bitwise manipulation but also lay the groundwork for further exploration into cryptographic and low-level programming concepts.

For further reading and a deeper understanding, explore the [Circular Shift](https://en.wikipedia.org/wiki/Circular_shift) and [Bitwise Operators](https://www.geeksforgeeks.org/python-bitwise-operators/) articles.


## 1. [rotl(x, n=1)](https://en.cppreference.com/w/cpp/numeric/rotl)
   - **Purpose:** Rotates the bits of a 32-bit unsigned integer `x` to the left by `n` positions.
   - **Method:** Combine a left shift and a right shift using bitwise OR, ensuring the wrapped-around bits are correctly placed.
   - **Considerations:** Use a bitmask (`0xFFFFFFFF`) to maintain 32-bit constraints, and handle cases where `n >= 32` by reducing `n` modulo 32.

In [69]:
def rotl(x, n=1):
    # How many bits we need to represent x (at least 1)
    width = x.bit_length() or 1
    mask = (1 << width) - 1 # For example if width is 7, mask is 0b1111111

    # Make sure n is in the range [0, width]
    n %= width

    # Do the rotation:
    # - shift x to the left by n bits
    # - bitwise AND with the mask to keep only the width least significant bits
    # - bitwise OR with the bits that were "shifted out" to the left
    return ((x << n) & mask) | (x >> (width - n))

# Example
x = 0b1010101 # 85
rotated = rotl(x) # rotate left by 1

# Print the binary representation of x and the rotated value, with leading zeros
width = x.bit_length() or 1
print("Original:", format(x, '0{}b'.format(width)))
print("Rotated: ", format(rotated, '0{}b'.format(width)))


Original: 1010101
Rotated:  0101011


## 2. [rotr(x, n=1)](https://en.cppreference.com/w/cpp/numeric/rotr)
   - **Purpose:** Rotates the bits of a 32-bit unsigned integer `x` to the right by `n` positions.
   - **Method:** Similar to `rotl`, but with right and left shifts interchanged.
   - **Considerations:** Maintain 32-bit integrity using a bitmask and modulo operations on `n`.

In [70]:
def rotr(x, n=1):
    width = x.bit_length() or 1 # How many bits we need to represent x (at least 1)
    mask = (1 << width) - 1  # For example if width is 7, mask is 0b1111111
    x &= mask # Make sure x is in the range [0, 2**width)
    n %= width # Make sure n is in the range [0, width)
    return (x >> n) | ((x << (width - n)) & mask) # Do the rotation

# Example
x = 0b0101010 # 42
rotated = rotr(x) # rotate left by 1

# Print the binary representation of x and the rotated value, with leading zeros
width = x.bit_length() or 1
print("Original:", format(x, '0{}b'.format(width)))
print("Rotated: ", format(rotated, '0{}b'.format(width)))

Original: 101010
Rotated:  010101


## 3. [ch(x, y, z)](https://crypto.stackexchange.com/questions/5358/what-does-maj-and-ch-mean-in-sha-256-algorithm)
   - **Purpose:** Implements a bitwise "choice" function where for each bit position, if the corresponding bit in `x` is `1`, the output takes the bit from `y`; otherwise, it takes the bit from `z`.
   - **Method:** Utilize bitwise operators to combine the bits from `y` and `z` based on `x`.
   - **Application:** This function is commonly used in cryptographic algorithms to blend bits in a controlled manner.

In [71]:
def ch(x, y, z):
    # (x AND y) XOR (NOT x AND z)
    return (x & y) ^ (~ x & z)

# Example
x = 0b1010 # 10
y = 0b0000 # 0  
z = 0b1111 # 15

result = ch(x, y, z)
print("Expeted: 0b101")
print("Result:", bin(result))

Expeted: 0b101
Result: 0b101


## 4. [maj(x, y, z)](https://crypto.stackexchange.com/questions/5358/what-does-maj-and-ch-mean-in-sha-256-algorithm)
   - **Purpose:** Computes the majority vote of the bits in `x`, `y`, and `z` for each bit position. The output bit is `1` if at least two of the three corresponding bits are `1`.
   - **Method:** Combine bitwise ANDs and ORs to efficiently determine the majority.
   - **Application:** This function is another staple in cryptographic hash functions, ensuring robust diffusion of input bits.

In [72]:
def maj(x, y, z):
    # (x AND y) XOR (x AND z) XOR (y AND z)
    return (x & y) ^ (x & z) ^ (y & z)

# Example
x = 0b1010 # 10
y = 0b0000 # 0
z = 0b1111 # 15

print("Expected:", bin(x))
print("Result:  ", bin(maj(x, y, z)))

Expected: 0b1010
Result:   0b1010


[Back to Top](#table-of-contents)

---

# **Task 2: Hash Functions**

In this task, we will convert a classic C hash function from *The C Programming Language* by Brian Kernighan and Dennis Ritchie into Python. The original hash function uses the `ord()` function (in Python, this returns the Unicode code point for a given character) to process each character in a string. For further details on the `ord()` function, see the [ord() Function documentation](https://www.w3schools.com/python/ref_func_ord.asp).

## The Original C Hash Function

The C code for the hash function is as follows:

```c
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}


### Explanation:
- **Loop through the string:** The function iterates over each character in the input string `s`.
- **Calculate hash value:** For every character, the hash value is updated by multiplying the current hash value by `31` and then adding the ASCII value of the character. This multiplication factor of ``31`` is chosen because it is a small prime number that helps in spreading out the hash values over a wide range.
- **Modulo Operation:** The final hash value is reduced using a modulo operation with `101`, another prime number, which helps in distributing the hash values more uniformly across available buckets (this can be especially useful for hash tables).

For more details on the concept of hash sizes and the reasoning behind such choices, refer to [ Hash Size in The C Programming Language.](https://www.cimat.mx/ciencia_para_jovenes/bachillerato/libros/%5BKernighan-Ritchie%5DThe_C_Programming_Language.pdf)

In [73]:
def hash(s, base=31, modulus=101):
    hashValue = 0
    for c in s:
        hashValue = ord(c) + base * hashValue # ord(c) returns the ASCII value of c
    return hashValue % modulus 

### Porting to Python
To implement the hash function in Python:

- **Using [`ord()`](https://www.w3schools.com/python/ref_func_ord.asp):** The `ord()` function is used to obtain the integer representation (Unicode code point) of each character in the string.

- **Iteration over the string:** We can iterate over the string directly since Python strings are iterable.

- **Modulo Operation:** The modulo operation `(% 101)` is applied at the end to get the final hash value.

In [74]:
# Some test strings
test_strings = [
    "hello", "world", "foo", "bar", "baz", "hash", "collision", "test",
    "abcd", "abce", "abcf", "java", "python", "rust", "c++", "javascript"
]

for string in test_strings:
    print(f"{string!r} -> {hash(string):>08b}, {hash(string)}")

'hello' -> 00010001, 17
'world' -> 00100010, 34
'foo' -> 01000101, 69
'bar' -> 00100100, 36
'baz' -> 00101100, 44
'hash' -> 00001111, 15
'collision' -> 00001000, 8
'test' -> 01010110, 86
'abcd' -> 01100100, 100
'abce' -> 00000000, 0
'abcf' -> 00000001, 1
'java' -> 01011101, 93
'python' -> 01011011, 91
'rust' -> 00010001, 17
'c++' -> 00111100, 60
'javascript' -> 00101011, 43


*Collison in `rust -> 17` and `hello -> 17`*

## Why `31` and `101`

After testing various candidates—such as bases `[31, 37, 257]` and moduli `[101, 127, 10_007]`—the values `31` and `101` have emerged as effective choices. As highlighted in Joshua Bloch's [*Effective Java*](https://ia800308.us.archive.org/16/items/java_20230528/Joshua%20Bloch%20-%20Effective%20Java%20%283rd%29%20-%202018.pdf):

* The value `31` was chosen because it is an odd prime. If it were even and the multiplication overflowed, information would be lost, because multiplication by `2` is equivalent to shifting. The advantage of using a prime is less clear, but it is traditional.  
* A nice property of `31` is that the multiplication can be replaced by a shift and a subtraction for better performance on some architectures: `31 * i == (i << 5) - i`. Modern VMs do this sort of optimization automatically.
This allows faster computation on certain architectures.
* Since `31 < 32`, it stays within the bit-width of a 32-bit processor.

Using `31` as the multiplier helps to generate a well-distributed hash by combining the contributions of each character in a string. Its odd prime nature avoids the pitfalls of even multipliers—such as losing information on overflow—and the computational shortcut (shifting and subtracting) can improve performance.

Similarly, the modulus `101` is chosen because it is a prime number. A prime modulus helps minimize collisions by ensuring a more uniform distribution of hash values. While the exact advantage of using a prime modulus like `101` might be less pronounced than that of the multiplier, it remains a traditional choice that has been proven effective in practice.

In summary, the combination of `31` and `101` has been historically favored in hash function design due to their mathematical properties and performance benefits, which remain relevant even as testing with other numbers (e.g., `37`, `257` for the base and `127`, `10_007` for the modulus) might show similar behavior in certain scenarios.


[Back to Top](#table-of-contents)

---

# **Task 3: SHA256 Padding**

The task is to implement a Python function that computes the SHA256 padding for a given file. This involves reading the file's contents in binary mode, calculating the padding according to the SHA256 ([SHA-2 - Wikipedia](https://en.wikipedia.org/wiki/SHA-2)) specification as detailed in [NIST FIPS 180-4](https://doi.org/10.6028/NIST.FIPS.180-4), and outputting the resulting padding in hexadecimal format.

In the SHA256 algorithm, before the input message is processed, it must be padded to satisfy the requirement that its total length (in bits) is equal to 448 bytes modulo 512. This is achieved by appending the following to the original message:
- **A single `1` bit:** Represented in hexadecimal as `0x80` (binary: `10000000`).
- **A series of `0` bits:** Enough zeros are appended so that the length of the message (after adding `0x80` and the zeros) is 56 bytes modulo 64. This ensures that, once an 8-byte (64-bit) representation of the original message length is appended, the overall message length is a multiple of 64 bytes (512 bits).
- **The original message length:** The length (in bits) of the original input is appended as a 64-bit big-endian integer.

This process is described in detail in [NIST FIPS 180-4](https://doi.org/10.6028/NIST.FIPS.180-4), which is the official document for the Secure Hash Standard (SHS). Additional context can be found on the [SHA-2 - Wikipedia](https://en.wikipedia.org/wiki/SHA-2) Wikipedia page and in [RFC 6234](https://datatracker.ietf.org/doc/html/rfc6234), which provide overviews and implementation notes for SHA-256.

In summary, the implementation should:
- Open and read the file in binary mode.
- Append `0x80` to the file's contents to indicate the start of the padding.
- Calculate the number of zero bytes needed so that, with the additional 8-byte length field, the total length is a multiple of 64 bytes.
- Append the original message length (in bits) as a 64-bit big-endian integer.


In [75]:
def sha256_padding(file_path):
    # Read the file contents in binary mode.
    with open(file_path, 'rb') as f:
        data = f.read()
    
    # Compute the original message length in bits.
    original_length = len(data)
    original_bit_length = original_length * 8

    # Step 1: Append the '1' bit as 0x80.
    # This represents the bit pattern 10000000.
    padding = b'\x80'

    # Step 2: Calculate how many zero bytes are needed.
    # After appending 0x80, the message should be padded so that its length (in bytes)
    # is congruent to 56 modulo 64. This leaves room for the final 8-byte length field.
    # That is, we need: (original_length + 1 + pad_len) % 64 == 56.
    pad_len = (56 - (original_length + 1) % 64) % 64
    padding += b'\x00' * pad_len

    # Step 3: Append the original length as a 64-bit big-endian integer.
    padding += struct.pack('>Q', original_bit_length)

    # Return the padding as a hexadecimal string.
    return padding.hex()

# Example
file_path = 'abc.txt'
test_answer = "80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018" # copied from task.md
padding = sha256_padding(file_path)
print("Padding abc.txt: " + padding)
print(padding == test_answer)

Padding abc.txt: 80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
True


### **How it works**

1. **Reading the file:**
    * file is opened using binary mode (i.e., "rb"). 
    * entire content of the file is read into `data`. 
    * length of the file (in bytes) is stored in a variable (let’s call it `L`)

2. **Appending the `1` Bit:**
    In SHA256 (and other SHA-2 algorithms), the first step of padding is to signal the end of the original message. This is done by appending a single `1` bit to the message.Although only a single bit is needed, bytes are the smallest addressable unit in computer memory. Thus, the 1 bit is appended as part of a full byte. The byte `0x80` in hexadecimal corresponds to the binary value `10000000`. After appending 0x80, the new message consists of the original data followed by this byte.

3. **Zero Padding:**

   We then calculate how many zero bytes are needed so that the total length (original message + `0x80` + zeros) is 56 bytes modulo 64. This ensures that when we later add the 8-byte length, the full message length becomes a multiple of 64 bytes (512 bits).

   * Let `L` be the original length (in bytes).
   * After appending `0x80`, the length becomes `L + 1`.
   * The formula used is:

     ```python
     pad_len = (56 - (L + 1) % 64) % 64
     ```
    This ensures:
    * If `(L + 1)` already leaves a remainder of 56 when divided by 64, no additional zero bytes are required.
    * Otherwise, it computes exactly how many 0x00 bytes are needed to reach the target length of 56 modulo 64.


4. **Appeding the Message Length:**

    The SHA256 padding requires that the last 8 bytes of the padded message represent the length of the original message in bits. This fixed-size representation ensures that the total message length is accurately recorded. Since the original length is in bytes (L), multiplying it by 8 converts it to bits (L * 8). This length is then represented as a 64-bit (8-byte) big-endian integer. In big-endian order, the most significant byte is placed first, which is crucial for consistency across different systems and aligns with the SHA256 specification.

    In Python, this conversion can be done using `struct.pack('>Q', L * 8)`, where '[>Q'](https://docs.python.org/3/library/struct.html) denotes an unsigned 64-bit big-endian integer. Alternatively, you can use `int.to_bytes(8, 'big')` on the computed bit length. This 8-byte representation is appended to the padded message, ensuring that the overall length of the padded message is a multiple of 64 bytes.

[Back to Top](#table-of-contents)

---

# **Task 4: Prime Numbers**
## Introduction

Prime numbers have long been a subject of fascination for mathematicians. A prime number is a natural number greater than 1 that has no positive divisors other than 1 and itself. They form the “building blocks” of integers because every integer can be uniquely factored into primes—a principle known as [the Fundamental Theorem of Arithmetic](https://en.wikipedia.org/wiki/Fundamental_theorem_of_arithmetic). Beyond their theoretical importance, prime numbers are also central to modern cryptography, particularly in secure communication systems like RSA. 

In this task, we will compute the first 100 prime numbers using two different algorithms. We’ll explain how each method works, discuss potential optimizations, and compare their performance.


### [Optimized Trial Division Method - O(√n) Time and O(1) Space](https://www.geeksforgeeks.org/check-for-prime-number/)
Every integer can be written as 6 multiplied by some number `𝑘` plus a remainder `𝑖` (where`𝑖` is 0, 1, 2, 3, 4, or 5). If `𝑖` is 0, 2, 3, or 4, then the number is divisible by 2 or 3. Since primes greater than 3 cannot be divisible by 2 or 3, they must leave a remainder of either 1 or 5 when divided by 6 (i.e. they are of the form `6𝑘+1 or 6k+5 `). This means that when checking if a number is prime, we only need to test numbers in these two forms, which cuts down on the number of checks.

**Why Not the Optimized Trial Division Method?**  
Although the optimized trial division method has a time complexity of O(√n) and minimal space overhead, it is not ideal for generating large lists of primes because:
- **Individual Checks:**  
  It checks each candidate number individually for primality, which becomes increasingly inefficient as numbers grow larger.
- **Computational Expense:**  
  For generating a list of primes, this approach is computationally expensive compared to sieve methods, which eliminate many non-prime candidates in a single pass.
- **Suitability:**  
  The trial division method is better suited for testing the primality of single numbers rather than for bulk generation, where sieve methods (like Eratosthenes and Sundaram) offer superior performance.

## Algorithm 1: Sieve of Eratosthenes
The Sieve of Eratosthenes is an efficient algorithm to generate all prime numbers up to a given limit. 

### How it works:

1. **Initialization:** Creating a boolean list where each index represents a number. Initially, all numbers are assumed to be prime (set to True), except for 0 and 1.
2. **Sieving Process:** Starting from the first prime (2), the algorithm marks all multiples of each prime as non-prime (False). The process continues with the next number that is still marked as prime.
3. **Collection:** After processing up to the square root of the limit, the remaining indices marked as True correspond to prime numbers.

Since the 100th prime is 541, choosing a limit of 600 ensures that we capture at least 100 primes.

For more details on the algorithm, you can refer [the Sieve of Eratosthenes.](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes)

In [76]:
def sieve_of_eratosthenes(limit):
    # Create a list of boolean values, initially set to True.
    is_prime = [True] * (limit + 1)
    # 0 and 1 are not prime numbers.
    is_prime[0] = is_prime[1] = False

    # Iterate from 2 up to the square root of the limit.
    for i in range(2, int(limit ** 0.5) + 1):
        if is_prime[i]:
            # Mark multiples of i as non-prime.
            for j in range(i * i, limit + 1, i):
                is_prime[j] = False

    # Return the list of numbers that are prime.
    return [num for num, prime in enumerate(is_prime) if prime]

# Since the 100th prime is 541, we set the limit to 600 to ensure we have at least 100 primes.
primes = sieve_of_eratosthenes(542)

print("The first", len(primes), "prime numbers are (Sieve of Eratosthenes):")
print(primes)

The first 100 prime numbers are (Sieve of Eratosthenes):
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


## Algorithm 2: Sieve of Sundaram
The Sieve of Sundaram is a prime number sieve that generates all prime numbers less than 2n + 2.

### How it works:
1. Create a list of integers from 1 to n (where n is an upper bound derived from the required number of primes).
2. Eliminate numbers of the form `i + j + 2ij` where `1 <= i <= j` and `i + j + 2ij <= n`.
3. The remaining numbers, when transformed into `2x + 1`, represent the prime numbers.

The advantage of this sieve is that it eliminates non-primes efficiently without requiring division

In [77]:
def sieve_of_sundaram(limit):
    n = (limit - 1) // 2  # We are interested in primes < 2n + 2
    sieve = [True] * (n + 1)
    
    for i in range(1, n + 1):
        for j in range(i, (n - i) // (2 * i + 1) + 1):
            sieve[i + j + 2 * i * j] = False  # Mark composite numbers
    
    primes = [2] + [2 * x + 1 for x in range(1, n + 1) if sieve[x]]  # Convert indices to primes
    return primes[:limit]

# Since the 100th prime is 541, we set the limit to 542 to ensure we have at least 100 primes.
first_100_primes = sieve_of_sundaram(542)

print("The first", len(first_100_primes), "prime numbers are (Sieve of Sundaram):")
print(first_100_primes)

The first 100 prime numbers are (Sieve of Sundaram):
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


## Comparison of Prime Number Sieves

Various algorithms have been developed to efficiently generate prime numbers. Three notable sieves are:

1. [**Sieve of Eratosthenes**](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes): An ancient algorithm that iteratively marks the multiples of each prime number starting from 2. The numbers that remain unmarked are prime.

2. [**Sieve of Sundaram**](https://en.wikipedia.org/wiki/Sieve_of_Sundaram): A sieve that eliminates numbers of the form `i + j + 2ij` (where `1 ≤ i ≤ j`) from a list of integers, then transforms the remaining numbers to obtain primes.


### Performance Comparison

| Sieve Algorithm           | Time Complexity       | Space Complexity | Pros                                         | Cons                                               |
|---------------------------|-----------------------|------------------|----------------------------------------------|----------------------------------------------------|
| **Sieve of Eratosthenes** | $O(n \log \log n)$  | $O(n)$         | Simple implementation; efficient for moderate ranges. | Less efficient for very large numbers due to memory constraints. |
| **Sieve of Sundaram**     | $O(n \log n)$       | $O(n)$         | Operates only on odd numbers, reducing space usage. | Requires additional computation to generate the final list of primes. |


### Detailed Analysis

- **Sieve of Eratosthenes**: This algorithm is straightforward and effective for generating all primes up to a moderate limit. Its simplicity makes it a common choice for many applications. However, its performance can degrade for very large numbers due to increased memory usage.

- **Sieve of Sundaram**: By focusing solely on odd numbers, this sieve reduces the number of operations and space required. After eliminating numbers of the form `i + j + 2ij`, the remaining numbers are transformed to obtain the primes. While it offers some efficiency benefits, it involves extra steps to produce the final list of primes.

### Conclusion

- **For small to medium-sized prime lists**, the **Sieve of Eratosthenes** is often preferred due to its simplicity and adequate performance.

- **In memory-constrained environments**, the **Sieve of Sundaram** offers a space-efficient alternative by operating only on odd numbers.

### Additional References

- [Harahap, M. K., & Khairina, N. (2019). *The Comparison of Methods for Generating Prime Numbers between The Sieve of Eratosthenes, Atkins, and Sundaram*. SinkrOn, 3(2), 293-298. ](https://jurnal.polgan.ac.id/index.php/sinkron/article/view/10129)

- ["Eratosthenes/Sundaram/Atkins Sieve Implementation in C." CodeProject. ](https://www.codeproject.com/Articles/490085/Eratosthenes-Sundaram-Atkins-Sieve-Implementation)


[Back to Top](#table-of-contents)

---

# Task 5: Roots

Calculate the first 32 bits of the fractional part of the square roots of the first 100 prime numbers.

In this guide, we break down the `get_fractional_bits()` function linearly. The function calculates the first 32 bits of the fractional part of the square root of a given number. We'll demonstrate each step using a specific prime number (in this case, 257) as an example.

In [78]:
# Setup: Assign a prime number for testing
test_prime = 257

## Step 1: Calculate the Square Root

We begin by calculating the square root of the given number using Python's `math.sqrt()` function.

In [79]:
# Step 1: Compute the square root of the number
root = math.sqrt(test_prime)
print("Square root of", test_prime, "is", root)
# Expected output: Square root of 2 is approximately 16.0312195418814

Square root of 257 is 16.0312195418814


## Step 2: Extract the Fractional Part

Next, we subtract the integer portion of the square root (using `math.floor()`) to isolate the fractional part.

In [80]:
# Step 2: Isolate the fractional part by subtracting the floor of the root
fractional_part = root - math.floor(root)
print("Fractional part of", root, "is", fractional_part)
# Expected output: Fractional part of ~16.0312195418814 is 0.031219541881398527

Fractional part of 16.0312195418814 is 0.031219541881398527


## Step 3: Shift the Fractional Bits

To capture the first 32 binary digits, multiply the fractional part by 2 raised to the power of 32.

In [81]:
# Step 3: Multiply the fractional part by 2^32 to shift the bits into the integer part
bits = 32
shifted_value = fractional_part * (2 ** bits)
print("Shifted value:", shifted_value)
# The shifted value is a float where its integer part corresponds to the first 32 binary digits.

Shifted value: 134086911.37670898


## Step 4: Convert to an Integer

Convert the shifted value to an integer. This step discards any remaining fractional bits beyond the first 32.

In [82]:
# Step 4: Convert the shifted value to an integer
int_value = int(shifted_value)
print("Integer value representing the first 32 bits:", int_value)
# The integer value now holds the bit pattern of the fractional part.

Integer value representing the first 32 bits: 134086911


## Step 5: Format as a 32-bit Binary String

Finally, format the integer into a binary string that is exactly 32 bits long, padding with zeros if necessary.


In [83]:
# Step 5: Format the integer as a binary string padded to 32 bits
binary_string = format(int_value, f'0{bits}b')
print("32-bit binary representation:", binary_string)
# Expected output: A 32-character binary string, for example "01101001011010101100101010101111"


32-bit binary representation: 00000111111111100000000011111111


## Algorithm Summary

1. **Compute the square root** of the given number.
2. **Extract the fractional part** by subtracting the floor of the square root.
3. **Shift the fractional bits** to the left by multiplying by 2^32.
4. **Convert the shifted value** to an integer, discarding extra fractional bits.
5. **Format the integer** into a 32-bit binary string with proper zero-padding.


## Testing the Complete Function

Below is the complete `get_fractional_bits()` function along with testing code using the prime number 2.


In [84]:
def get_fractional_bits(num, bits=32):
    # Step 1: Compute square root
    root = math.sqrt(num)
    
    # Step 2: Extract the fractional part
    fractional_part = root - math.floor(root)
    
    # Step 3: Shift the fractional part to extract the first 'bits' binary digits
    shifted_value = fractional_part * (2 ** bits)
    
    # Step 4: Convert the shifted value to an integer
    int_value = int(shifted_value)
    
    # Step 5: Format the integer as a 32-bit binary string (with leading zeros if needed)
    binary_string = format(int_value, f'0{bits}b')
    
    return binary_string

# Testing the function with the prime number 2
result = get_fractional_bits(test_prime)
print(f"First 32 bits of the fractional part of sqrt({test_prime}):", result)

First 32 bits of the fractional part of sqrt(257): 00000111111111100000000011111111


In [85]:
# Compute and store the 32-bit binary fractional part for each prime.
first_100_primes = sieve_of_sundaram(542)
fractional_bits = {prime: get_fractional_bits(prime) for prime in first_100_primes}
# First, sort the primes
sorted_primes = sorted(fractional_bits)
n = len(sorted_primes)

# Calculate the number of rows required (ceiling of n divided by 4)
rows = (n + 2) // 4  # +2 ensures proper ceiling for division by 4

# Define a format for printing three columns side by side.
row_format = "{:<5}| {:<35} || {:<5}| {:<35}|| {:<5}| {:<35}|| {:<5}| {:<35}"

# Print header for all three columns.
header = row_format.format("Prime", "32-bit Fractional Part", 
                           "Prime", "32-bit Fractional Part", 
                           "Prime", "32-bit Fractional Part",
                           "Prime", "32-bit Fractional Part")
print(header)

# Iterate over the number of rows and print each corresponding row from each column.
for i in range(rows):
    # Column 1: index i
    col1_prime = sorted_primes[i] if i < len(sorted_primes) else ""
    col1_bits = fractional_bits[col1_prime] if col1_prime != "" else ""
    
    # Column 2: index i + rows
    idx2 = i + rows
    col2_prime = sorted_primes[idx2] if idx2 < len(sorted_primes) else ""
    col2_bits = fractional_bits[col2_prime] if col2_prime != "" else ""
    
    # Column 3: index i + 2*rows
    idx3 = i + 2 * rows
    col3_prime = sorted_primes[idx3] if idx3 < len(sorted_primes) else ""
    col3_bits = fractional_bits[col3_prime] if col3_prime != "" else ""

    # Column 4: index i + 3*rows
    idx4 = i + 3 * rows
    col4_prime = sorted_primes[idx4] if idx4 < len(sorted_primes) else ""
    col4_bits = fractional_bits[col4_prime] if col4_prime != "" else ""
    
    print(row_format.format(str(col1_prime), col1_bits, 
                            str(col2_prime), col2_bits, 
                            str(col3_prime), col3_bits,
                            str(col4_prime), col4_bits))

Prime| 32-bit Fractional Part              || Prime| 32-bit Fractional Part             || Prime| 32-bit Fractional Part             || Prime| 32-bit Fractional Part             
2    | 01101010000010011110011001100111    || 101  | 00001100110001001010011000010001   || 233  | 01000011101010111001111110110110   || 383  | 10010010000001001100110110011101   
3    | 10111011011001111010111010000101    || 103  | 00100110000111011100000111110010   || 239  | 01110101101010011111100100011101   || 389  | 10111001000110111111011001100011   
5    | 00111100011011101111001101110010    || 107  | 01011000000101011010011110111110   || 241  | 10000110001100000101000000011001   || 397  | 11101100110000111000110010011101   
7    | 10100101010011111111010100111010    || 109  | 01110000101101111110110101100111   || 251  | 11010111110011011000000101110011   || 401  | 00000110011001010110000010010101   
11   | 01010001000011100101001001111111    || 113  | 10100001010100010011110001101001   || 257  | 0000011

[Back to Top](#table-of-contents)

---

# **Task 6: Proof of Work**
Find the word(s) in the English language with the greatest number of `0` bits at the beginning of their SHA256 hash digest.  
Include proof that any word you list is in at least one English dictionary.  

In this task, we must search through an English word list to find the word(s) whose SHA256 hash begins with the greatest number of consecutive 0 bits. I used a word list from [Word Lists](https://web.archive.org/web/20131118073324/https://www.infochimps.com/datasets/word-list-350000-simple-english-words-excel-readable) and verified that any candidate appears in an English dictionary – for example, see the entry for “goaltenders” in [Collins Dictionary](https://www.collinsdictionary.com/dictionary/english/goaltender#:~:text=Word%20forms%3A-,goaltenders,-countable%20noun).

## 1. Computing the Number of Leading 0 Bits in a SHA256 Digest
This function computes the SHA256 hash of a given word and then counts how many consecutive 0 bits appear starting from the most significant bit.

In [86]:
# hashlib is a built-in library in Python for hashing algorithms
word = "hello"
digest = hashlib.sha256(word.encode('utf-8')).digest()

# Convert digest to a single binary string
binary_str = ''.join(format(byte, '08b') for byte in digest)

# Cut after 12 bits for display
cut_str = binary_str[:64]

# Count leading zeros in the full digest
leading_zeros = len(binary_str) - len(binary_str.lstrip('0'))

print("First 12 bits of SHA256('hello') in binary:")
print(cut_str+"...")
print("\nLeading 0 bits in full digest:", leading_zeros)


First 12 bits of SHA256('hello') in binary:
0010110011110010010011011011101001011111101100001010001100001110...

Leading 0 bits in full digest: 2


### Using Hello

* The result is a 32-byte digest (since SHA256 produces a 256-bit hash).
* For example, the first byte in this digest is 0x2C (displayed as , in the byte string), which in binary is 00101100.
* This digest serves as the basis for our next step, where we count how many consecutive 0 bits appear from the beginning.

In [87]:
count = 0
found = False  # Flag to indicate when the first '1' is found

# Loop through each byte in the digest
for byte in digest:
    # Check each bit from most significant (bit 7) to least significant (bit 0)
    for i in range(8):
        if byte & (1 << (7 - i)):
            # As soon as a 1 is encountered, print the count and exit both loops
            found = True
            break
        else:
            count += 1
    if found:
        break
    
# Print the number of leading zeros
print(f"Number of leading 0 bits in SHA256('{word}'): {count}")

Number of leading 0 bits in SHA256('hello'): 2


### Function `leading_zero_bits(word)`

In [88]:
def leading_zero_bits(word):
    """
    Computes the SHA256 hash digest for a given word (UTF-8 encoded) and
    returns the number of consecutive 0 bits at the beginning of the digest.
    """
    # Compute the SHA256 digest.
    digest = hashlib.sha256(word.encode('utf-8')).digest()
    count = 0
    # Loop through each byte in the digest.
    for byte in digest:
        # Check each bit from most significant (bit 7) to least significant (bit 0)
        for i in range(8):
            if byte & (1 << (7 - i)):
                # As soon as a 1 is encountered, return the count.
                return count
            count += 1
    return count

# Example usage
word = "Digest"
count = leading_zero_bits(word)
print(f"Number of leading 0 bits in SHA256('{word}'): {count}")

Number of leading 0 bits in SHA256('Digest'): 1


### Counting Zeros:
* It then initializes a counter (count) to track the number of consecutive 0 bits.
* The function loops through each byte in the digest.
* For each byte, it examines every bit from the most significant (leftmost) to the least significant (rightmost) using a bit mask.

### Early Termination:
* When a bit with the value 1 is found, the function immediately returns the count of 0 bits encountered up to that point.
* For instance, if the first byte is 0x2C (binary 00101100), the function counts:
    * Bit 1: 0
    * Bit 2: 0
    * Bit 3: 1 (stop here)

Hence, for "hello", the function would return 2, since the digest starts with 2 consecutive 0 bits.

## 2. Loading an English Word List & Finding the Word(s) with Maximum Leading 0 Bits
This section iterates through the word list, computes the number of leading 0 bits for each word, and keeps track of the word(s) with the highest count.

In [89]:
# Try to use the infochimps file.
dictionary_path = 'words.txt'
try:
    with open(dictionary_path, 'r') as f:
        words = [line.strip() for line in f if line.strip()]
except FileNotFoundError:
    print(f"File {dictionary_path} not found.")
# Initialize variables to track the maximum number of leading zeros
max_zeros = -1
best_words = []

In [90]:
for word in words:
    # Consider only words that consist of alphabetic characters.
    if not word.isalpha():
        continue
    # Convert to lower-case to ensure consistency.
    zeros = leading_zero_bits(word.lower())
    # Update our best candidate(s) if this word has more leading zeros.
    if zeros > max_zeros:
        max_zeros = zeros
        best_words = [word]
    elif zeros == max_zeros:
        best_words.append(word)

print("Maximum leading 0 bits:", max_zeros)
print("Word(s) achieving that:", best_words)


Maximum leading 0 bits: 18
Word(s) achieving that: ['goaltenders']


### Proof of Work

In this task, I searched for English word(s) that produce the highest number of leading zero bits in their SHA-256 hash digest — a process analogous to the proof-of-work concept used in blockchain mining. After testing a comprehensive English word list, the word **"[goaltenders](https://www.collinsdictionary.com/dictionary/english/goaltender#:~:text=Word%20forms%3A-,goaltenders,-countable%20noun)
"** achieved the highest observed result, with **18 leading zero bits** in its hash digest.

The fact that 'goaltenders' naturally produced a SHA-256 digest with 18 leading zero bits demonstrates the probabilistic nature of hash functions; even without intentional manipulation, some inputs naturally yield digests with more leading zeroes due to randomness and uniform distribution. While 18 leading zero bits are far from cryptographic difficulty thresholds (e.g., 20+ bits typically used in mining puzzles), it highlights the usefulness of hash functions as computational puzzles where "work" can be verified simply by re-hashing the result.

The word **"goaltenders"** was verified to exist in a standard English dictionary, satisfying the task requirement that all results be valid words. The word list used was:  
[`words.txt`](https://web.archive.org/web/20131118073324/https://www.infochimps.com/datasets/word-list-350000-simple-english-words-excel-readable)) *(InfoChimps datasets `web.archive.org`)*  

This task helped illustrate both the properties of SHA-256 and the broader concept of computational difficulty through hash-based challenges.


[Back to Top](#table-of-contents)

---

# **Task 7: Turing Machines**
This task implements a simulated Turing Machine that adds `1` to a binary number encoded on its tape. Think of it like a crab that walks along the tape. The crab starts at the **left-most non-blank symbol** and treats the **right-most symbol as the least significant bit**, following the standard binary addition rules.

## Tape Behavior

- The tape contains a binary string (e.g., `100111`).
- The crab begins at the leftmost digit and moves right until it encounters a blank (`_`), which marks the end of the number.
- It then moves left to begin the addition process.

## Addition Logic

- Starting from the least significant bit (rightmost digit):
  - If the bit is `0`, it is changed to `1`, and the crab halts.
  - If the bit is `1`, it is changed to `0`, and the crab continues left (carrying the 1).
  - If the crab moves past the first digit (into a blank), it writes a `1` to account for the carry.

This logic simulates the process of adding one to a binary number, including proper handling of carries through multiple digits.

In [91]:
def crab_add_one(tape_string):
    # Add a blank at the front and end of the tape
    tape = ['_'] + list(tape_string) + ['_']
    crab_position = 1  # Start at the left-most non-blank symbol
    crab_mood = 'going_to_end'

    while True:
        if crab_mood == 'going_to_end':
            if tape[crab_position] in ['0', '1']:
                crab_position += 1
            else:
                crab_position -= 1
                crab_mood = 'adding'

        elif crab_mood == 'adding':
            if tape[crab_position] == '0':
                tape[crab_position] = '1'
                break
            elif tape[crab_position] == '1':
                tape[crab_position] = '0'
                crab_position -= 1
            elif tape[crab_position] == '_':
                tape[crab_position] = '1'
                break

    # Join the tape back into a string and strip leading blanks
    return ''.join(tape).strip('_')


# Test example
input_tape = "0101010"
output_tape = crab_add_one(input_tape)

print("Input Tape:  ", input_tape)
print("Output Tape: ", output_tape)
# Expected output: 0101011 

Input Tape:   0101010
Output Tape:  0101011


## Testing the Turing Machine

To validate the implementation of the Turing Machine that adds 1 to a binary number, a series of test cases were created. These test a range of conditions including basic addition, full carry-over, edge cases, and empty input.

In [92]:
def test_crab_add_one():
    test_cases = [
        # Format: (input, expected_output)
        ("0", "1"),
        ("1", "10"),
        ("10", "11"),
        ("11", "100"),
        ("100111", "101000"),
        ("1111", "10000"),
        ("101010", "101011"),
        ("000000", "000001"),
        ("", "1"),  # Edge case: empty input
        ("1" * 32, "1" + "0" * 32),  # Large input with full carry
    ]
    
    passed = True
    for binary_in, expected in test_cases:
        result = crab_add_one(binary_in)
        if result != expected:
            print(f">:( FAIL: Input '{binary_in}' → Expected '{expected}', Got '{result}'")
            passed = False
        else:
            print(f":D PASS: Input '{binary_in}' → Output '{result}'")

    if passed:
        print("\nAll tests passed.")
    else:
        print("\nSome tests failed.")

# Run the test function
test_crab_add_one()

:D PASS: Input '0' → Output '1'
:D PASS: Input '1' → Output '10'
:D PASS: Input '10' → Output '11'
:D PASS: Input '11' → Output '100'
:D PASS: Input '100111' → Output '101000'
:D PASS: Input '1111' → Output '10000'
:D PASS: Input '101010' → Output '101011'
:D PASS: Input '000000' → Output '000001'
:D PASS: Input '' → Output '1'
:D PASS: Input '11111111111111111111111111111111' → Output '100000000000000000000000000000000'

All tests passed.




### Test Case Explanations

| **Input**      | **Expected Output** | **Explanation**                                                                 |
|----------------|---------------------|----------------------------------------------------------------------------------|
| `0`            | `1`                 | Adding 1 to 0 results in 1.                                                    |
| `1`            | `10`                | Adding 1 to binary 1 produces 10.                                              |
| `10`           | `11`                | Binary 2 + 1 = 3 → `10` → `11`.                                                |
| `11`           | `100`               | Full carry: 1 + 1 = 0 (carry 1), next 1 + 1 = 0 (carry 1), then prepend 1.     |
| `100111`       | `101000`            | Given in the task — binary 39 + 1 = 40.                                        |
| `1111`         | `10000`             | Full 1s require carry through all digits → prepend a 1.                        |
| `101010`       | `101011`            | Only the last bit flips from 0 → 1.                                            |
| `000000`       | `000001`            | Only the last bit flips, no carry needed.                                      |
| *(empty)* `""` | `1`                 | Edge case: treat as 0, result should be 1.                                     |
| `111...1` (32) | `1` + 32 zeros      | Verifies proper carry logic through long input — `2³² - 1 + 1 = 2³²`.         |

These tests confirm that the Turing Machine logic handles:
- Normal addition without carry
- Multiple carries through 1s
- Full-width overflow and growth
- Leading/trailing zeros
- Empty tape edge case

[Back to Top](#table-of-contents)

---

# **Task 8: Computational Complexity**
This task investigates the computational complexity of **bubble sort** by measuring how many **comparisons** it makes when sorting every possible permutation of a five-element list:  

## Method

- A `bubble_sort_with_comparisons()` function was written to sort a list while counting how many comparisons it performs.
- All permutations (120 total) of the list `[1, 2, 3, 4, 5]` were generated using `itertools.permutations()` (from the standard library).
- Each permutation was sorted, and the number of comparisons was recorded.

In [93]:
from itertools import permutations

def bubble_sort_with_comparisons(arr):
    arr = list(arr)
    n = len(arr)
    comparisons = 0

    for i in range(n):
        swapped = False  # Track if any swaps occur in this pass
        for j in range(n - 1 - i):
            comparisons += 1
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
        if not swapped:
            break  # Exit early if the list is already sorted

    return comparisons


## Observations

Bubble sort's time complexity in terms of comparisons is always:

- **Worst Case:**  
  - When the list is in reverse order (`[5, 4, 3, 2, 1]`).
  - Comparisons: `10 + 9 + 8 + 7 + 6 = 40`.
- **Best Case:**  
  - When the list is already sorted (`[1, 2, 3, 4, 5]`).
  - Still performs all comparisons: 40 in this version (no early-exit optimization).

> (*) This implementation uses the standard bubble sort optimization of skipping sorted tail elements (`n - 1 - i`), **and** it includes an early-exit check to detect when the list is already sorted.

> In the best case (fully sorted input), the function performs only `n - 1 = 4` comparisons (i.e., one full pass) and exits early.

> In the worst case (fully reversed), it performs the full `10 + 9 + 8 + 7 + 6 = 40` comparisons.

In [94]:
# Generate results
results = []
for p in permutations([1, 2, 3, 4, 5]):
    perm = ''.join(map(str, p))
    comps = bubble_sort_with_comparisons(p)
    results.append((perm, comps))

# Adjustable column count
columns = 5  # ← change this to 2, 3, 4, etc.

# Prepare column chunks
rows_per_column = (len(results) + columns - 1) // columns  # ceiling division
columns_data = [results[i * rows_per_column:(i + 1) * rows_per_column] for i in range(columns)]

# Normalize column lengths
max_len = max(len(col) for col in columns_data)
for col in columns_data:
    while len(col) < max_len:
        col.append(("", ""))  # Empty row

# Print headers
header = ' | | '.join("Permutation  Comparisons" for _ in range(columns))
separator = ' | | '.join("-" * 24 for _ in range(columns))
print(header)
print(separator)

# Print aligned rows
for row_idx in range(max_len):
    row_cells = []
    for col in columns_data:
        perm, comps = col[row_idx]
        if perm:
            row_cells.append(f"{perm:<12} {comps:>10}")
        else:
            row_cells.append(" " * 24)
    print(' | | '.join(row_cells))

Permutation  Comparisons | | Permutation  Comparisons | | Permutation  Comparisons | | Permutation  Comparisons | | Permutation  Comparisons
------------------------ | | ------------------------ | | ------------------------ | | ------------------------ | | ------------------------
12345                 4 | | 21345                 7 | | 31245                 7 | | 41235                 7 | | 51234                 7
12354                 7 | | 21354                 7 | | 31254                 7 | | 41253                 9 | | 51243                 9
12435                 7 | | 21435                 7 | | 31425                 9 | | 41325                 9 | | 51324                 9
12453                 9 | | 21453                 9 | | 31452                10 | | 41352                10 | | 51342                10
12534                 7 | | 21534                 7 | | 31524                 9 | | 41523                 9 | | 51423                 9
12543                 9 | | 21543     

## Complexity Summary

The implementation of bubble sort includes two key optimizations:

1. **Inner loop reduction**: Skips comparisons over the already sorted tail (`range(n - 1 - i)`).
2. **Early exit**: Detects if a full pass completes without any swaps and exits early.

This results in improved performance, particularly for already sorted or nearly sorted inputs.

### Observed Behavior (n = 5):

| **Case Type**        | **Example Permutation** | **Comparisons** |
|----------------------|--------------------------|------------------|
| **Best Case**        | `12345`                  | 4                |
| **Common Case**      | (many permutations)      | 7–9              |
| **Worst Case**       | `54321`                  | 10               |

- The **best case** only requires 4 comparisons (a single pass).
- The **worst case** uses 10 comparisons — still far below the 40 comparisons a naive implementation would perform (i.e., `10 + 9 + 8 + 7 + 6 = 40`).
- Most permutations fall between 7–9 comparisons, showing how input order significantly impacts runtime.

This reflects **quadratic worst-case complexity** (O(n²)) but **linear best-case behavior** (O(n)) when the input is already sorted.

[Back to Top](#table-of-contents)

---

## **Conclusion**
...

[Back to Top](#table-of-contents)

---